In [43]:
import hashlib
import random

# Conversion of IAMRARE ID with MD5 Hashing

In [44]:
patient_ID = str(random.randint(0,9999))
print(f"Generated patient ID: {patient_ID}")

Generated patient ID: 2446


In [45]:
# Object creation
m = hashlib.md5()
m.update(b"{patient_ID}")

### Patient ID hashed to byte value

In [46]:
byte_value = m.digest()
print(f"Byte value: {byte_value}")


Byte value: b"('\xa7>\xb7i\xdf\xf049\x85\xfc\x88<\xaaJ"


### Byte value shown as hexadecimal

In [47]:
hex_value = m.hexdigest()
print(f"Hexadecimal value: {hex_value}")

Hexadecimal value: 2827a73eb769dff0343985fc883caa4a


### Patient ID to MD5Hash

In [48]:
import hashlib
import random

# Generate a random patient ID (0 to 9999) and convert it to a string
patient_ID = str(random.randint(0, 9999))
print(f"Generated patient ID: {patient_ID}")

# Create an MD5 hash object
m = hashlib.md5()

# Update the hash object with the patient ID (formatted as bytes)
m.update(patient_ID.encode())  # Properly encode the string as bytes

# Get the hashed value in byte format
byte_value = m.digest()
print(f"Byte value: {byte_value}")

# Get the hashed value in hexadecimal format
hex_value = m.hexdigest()
print(f"Hexadecimal value: {hex_value}")

# Essentially, this turns the patient ID into a fixed-length 128-bit (32-character) 
# hexadecimal hash, ensuring obfuscation while maintaining a consistent format.


Generated patient ID: 5927
Byte value: b'\xc8\xaf\xe8\x05\xc0\x97\xda\xb1\xf1\xe5\xbd\xd5\x7f\x8d)1'
Hexadecimal value: c8afe805c097dab1f1e5bdd57f8d2931


# AES 128 encryption key generation and connection with MD5 Hashed ID in IAMRARE

In [49]:
print(hex_value)

c8afe805c097dab1f1e5bdd57f8d2931


In [50]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

data = hex_value.encode()

print(data)

aes_key = get_random_bytes(16)
print(f"AES key: {aes_key.hex()}")

cipher = AES.new(aes_key, AES.MODE_CTR)
ciphertext = cipher.encrypt(data)
print(f"Cipher text: {ciphertext.hex()}")

b'c8afe805c097dab1f1e5bdd57f8d2931'
AES key: 9a0a358d2588aa10043f7553b25d5202
Cipher text: 73e6c22ed1c7fcc91af311113b4dd1f861e120fa6cf0517d7e166182168185dd


In [51]:
print(data.decode())

c8afe805c097dab1f1e5bdd57f8d2931


# Test Example 1 - AES_CTR
* Pros: Fast, no padding needed, parallelizable	<br>
* Cons: No authentication (use HMAC separately)


In [52]:
from Crypto.Cipher import AES
from Crypto.Hash import HMAC, SHA256
from Crypto.Random import get_random_bytes

data = 'secret data to transmit'.encode()

aes_key = get_random_bytes(16)
hmac_key = get_random_bytes(16)

cipher = AES.new(aes_key, AES.MODE_CTR)
ciphertext = cipher.encrypt(data)

hmac = HMAC.new(hmac_key, digestmod=SHA256)
tag = hmac.update(cipher.nonce + ciphertext).digest()

with open("encrypted.bin", "wb") as f:
    f.write(tag)
    f.write(cipher.nonce)
    f.write(ciphertext)

# Share securely aes_key and hmac_key with the receiver
# encrypted.bin can be sent over an unsecure channel

In [53]:
import sys
from Crypto.Cipher import AES
from Crypto.Hash import HMAC, SHA256

# Somehow, the receiver securely get aes_key and hmac_key
# encrypted.bin can be sent over an unsecure channel

with open("encrypted.bin", "rb") as f:
    tag = f.read(32)
    nonce = f.read(8)
    ciphertext = f.read()

try:
    hmac = HMAC.new(hmac_key, digestmod=SHA256)
    tag = hmac.update(nonce + ciphertext).verify(tag)
except ValueError:
    print("The message was modified!")
    sys.exit(1)

cipher = AES.new(aes_key, AES.MODE_CTR, nonce=nonce)
message = cipher.decrypt(ciphertext)
print("Message:", message.decode())

Message: secret data to transmit


# Test Example 2 - AES_CBC
* Pros: Secure, widely used	<br>
* Cons: Needs padding, needs HMAC for integrity


In [54]:
from base64 import b64encode, b64decode
from binascii import unhexlify

from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

iv = "7bde5a0f3f39fd658efc45de143cbc94"
password = "3e83b13d99bf0de6c6bde5ac5ca4ae68"
# msg = "this is a message"
msg = hex_value

print(f"IV: {iv}")
print(f"PWD: {password}")
print(f"MSG: {msg}")

# Convert Hex String to Binary
iv = unhexlify(iv)
password = unhexlify(password)

# Pad to AES Block Size
msg = pad(msg.encode(), AES.block_size)
# Encipher Text
cipher = AES.new(password, AES.MODE_CBC, iv)
cipher_text = cipher.encrypt(msg)

# Encode Cipher_text as Base 64 and decode to String
out = b64encode(cipher_text).decode('utf-8')
print(f"OUT: {out}")

# Decipher cipher text
decipher = AES.new(password, AES.MODE_CBC, iv)
# UnPad Based on AES Block Size
plaintext = unpad(decipher.decrypt(b64decode(out)), AES.block_size).decode('utf-8')
print(f'PT: {plaintext}')

IV: 7bde5a0f3f39fd658efc45de143cbc94
PWD: 3e83b13d99bf0de6c6bde5ac5ca4ae68
MSG: c8afe805c097dab1f1e5bdd57f8d2931
OUT: tSPSn0Scdj2uEsGaNkvHpWvp+uYCq27FQI0PUJzL3DHiOXeCjw3ohg6oegOFFQ6e
PT: c8afe805c097dab1f1e5bdd57f8d2931


# Test Example 3 - AES_GCM THIS 
# SEEMS LIKE THE BEST FOR CURRENT USE CASE
* Pros: Best security, authenticated, no padding needed	<br>
* Cons: Slightly more complex


In [55]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

# This module is for converting ASCII characters ⇄ binary64. 
# Base64 is NOT encryption. It makes binary data text-friendly.
from base64 import b64encode, b64decode

# Generate a random 16-byte key
key = get_random_bytes(16)


# #NOTE: .encode() and .decode() handle converting strings ⇄ bytes for encryption & decryption.


def encrypt(plaintext, key):
    # Generate a random 12-byte nonce
    
    nonce = get_random_bytes(12)
    
    # Create AES-GCM cipher object
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    
    # Encrypt and generate authentication tag
    ciphertext, tag = cipher.encrypt_and_digest(plaintext.encode())

    # Encode everything as Base64 for easy storage/transmission
    return b64encode(nonce + ciphertext + tag).decode()

def decrypt(encrypted_text, key):
    # Decode Base64
    encrypted_data = b64decode(encrypted_text)

    # Extract nonce, ciphertext, and tag

    # #NOTE: A Nonce (Number Used Once) is a random or unique value that is used only once in a cryptographic operation.
    # #NOTE: It prevents replay attacks and ensures that the same plaintext encrypted multiple times produces different ciphertexts.
    nonce = encrypted_data[:12]
    
    ciphertext = encrypted_data[12:-16]

    # #NOTE: The authentication tag (tag) ensures data integrity and authenticity.
    # #NOTE: It verifies that the encrypted data has not been altered or tampered with.
    # #NOTE: If the tag does not match during decryption, an error will be raised, preventing unauthorized modifications.
    tag = encrypted_data[-16:]
    
    # Decrypt using the same nonce
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    plaintext = cipher.decrypt_and_verify(ciphertext, tag)
    
    return plaintext.decode()

# Example Usage
message = "Hello, this is a secret message!"
encrypted_message = encrypt(message, key)
decrypted_message = decrypt(encrypted_message, key)

print("Encrypted:", encrypted_message)
print("Decrypted:", decrypted_message)


Encrypted: 1kUeS4RsdoJ0Pqx9Aj11QpbTruDAgJV26bxmJ+0WO/x1bV78X3oW6WUeA6l0aDFidzhRTm+KhS1DWNeA
Decrypted: Hello, this is a secret message!


# FINAL PIPELINE TEST:

## 1. Save Patient ID, Hashed ID, and Key to a TSV File


In [56]:
import hashlib
import random
from Crypto.Random import get_random_bytes
from base64 import b64encode
import os

# Generate a random participant ID (0 to 9999) and convert it to a string
participant_ID = str(random.randint(0, 9999))
print(f"Generated participant ID: {participant_ID}")

# Step 1: Hash using MD5
m = hashlib.md5()
m.update(participant_ID.encode())  # Encode the string as bytes
hashed_participant_id = m.hexdigest()
print(f"Hashed participant ID: {hashed_participant_id}")

# Generate a random 16-byte AES key
key = get_random_bytes(16)

##### SAVE DATA FOR DEMO ######

# File path for saving data
participant_data_file = 'participant_data.tsv'

# Check if the file exists, if not, write headers
if not os.path.exists(participant_data_file):
    with open(participant_data_file, 'w') as file:
        file.write("Participant ID\tHashed ID\tAES Key (Base64)\n")  # Writing headers

# Save Participant ID, Hashed ID, and AES Key to a TSV file
with open(participant_data_file, 'a') as file:  # Open in append mode
    file.write(f"{participant_ID}\t{hashed_participant_id}\t{b64encode(key).decode()}\n")


Generated participant ID: 7705
Hashed participant ID: 76d4110e944e83212bafa4b11ebf2b7e


In [57]:
print(f'The AES key: {b64encode(key).decode()}')

The AES key: CK3xIClQ3TbC1QMEAkCPwA==


## 2. Encrypt the Hashed Patient ID and Save to a Different TSV File



In [58]:
from Crypto.Cipher import AES
from base64 import b64encode

# Encrypt function as defined earlier
def encrypt(plaintext, key):
    # Generate a random 12-byte nonce (for AES-GCM)
    nonce = get_random_bytes(12)
    print(f"Nonce:{nonce}")

    # Create AES-GCM cipher object
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    
    # Encrypt and generate authentication tag
    ciphertext, tag = cipher.encrypt_and_digest(plaintext.encode())
    print(f"Ciphertext:{ciphertext}")
    print(f"Tag:{tag}")
    # Encode everything as Base64 for easy storage/transmission
    return b64encode(nonce + ciphertext + tag).decode()

# Encrypt the Hashed participant ID
encrypted_participant_id = encrypt(hashed_participant_id, key)
print(f"Encrypted (AES) participant ID: {encrypted_participant_id}")

# File path for saving encrypted data
encrypted_participant_data_file = 'encrypted_participant_data.tsv'

# Check if the file exists, if not, write headers
if not os.path.exists(encrypted_participant_data_file):
    with open(encrypted_participant_data_file, 'w') as file:
        file.write("Hashed ID\tEncrypted Participant ID\n")  # Writing headers

# Save Hashed ID and Encrypted Participant ID to a TSV file
with open(encrypted_participant_data_file, 'a') as file:  # Open in append mode
    file.write(f"{hashed_participant_id}\t{encrypted_participant_id}\n")


Nonce:b'Q6<\xa1B\x84\x90\x96\x14\x01\x1c\xe9'
Ciphertext:b"\x10yw\xf4\xfbY`\x97\xe5\xdayJXA\xb7\xc9\xa8\xb0\xa8C\xd4\x13\xe7\x92\xaf0\xc6\x89\xe9c'\xa2"
Tag:b'\xa5\xfb\xc8)\xe2+\xec\x9f\xb3\xa7\x82\xab\x1b@\x89\xd6'
Encrypted (AES) participant ID: UTY8oUKEkJYUARzpEHl39PtZYJfl2nlKWEG3yaiwqEPUE+eSrzDGieljJ6Kl+8gp4ivsn7OngqsbQInW


In [59]:
len("mnVIdmDpoO/TLshRApY6VQ==")
len('uoo2MIbVPQ7ko2WzI79GRy+bvT0UN76AYHrnnpDfcxEso/91535bT1mRdEhhmnigIGeyxOFs2n5XthT1')

80

In [60]:
print(len(key.hex()))
print(f"Nonce:{nonce.hex()}")
print(f"Ciphertext:{ciphertext.hex()}")
tag = b'8+!\x16/\xb5]\xc2\xcauBX%\xb8\xb3h'
print(f"Tag:{tag.hex()}")

32
Nonce:f7b94b5e4f8eafa3
Ciphertext:e558f89c78014bb3983c40a17c62db30b744bacecaae74
Tag:382b21162fb55dc2ca75425825b8b368


## 3. Decrypt the Encrypted Participant ID. <br>
## Asks for 2 inputs: AES Key and Encrypted Participant ID
## This is an terminal interactive script



In [65]:
from Crypto.Cipher import AES
from base64 import b64decode

# Decrypt function as defined earlier
def decrypt(encrypted_text, key):
    # Decode Base64
    encrypted_data = b64decode(encrypted_text)

    # Extract nonce, ciphertext, and tag
    nonce = encrypted_data[:12]
    ciphertext = encrypted_data[12:-16]
    tag = encrypted_data[-16:]

    # Decrypt using the same nonce
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    plaintext = cipher.decrypt_and_verify(ciphertext, tag)

    return plaintext.decode()

# Get AES key input from the user
key_input = input("Enter the AES key (Base64 encoded): ")
key = b64decode(key_input)  # Decode the input key from Base64

# Get the encrypted participant ID input from the user
encrypted_participant_id_input = input("Enter the encrypted participant ID (Base64 encoded): ")

# Decrypt the encrypted participant ID
decrypted_participant_id = decrypt(encrypted_participant_id_input, key)

# Print the decrypted participant ID
print(f"Initial MD5 Hash Input (Hashed Participant ID): {hashed_participant_id}")
print(f"Decrypted Participant ID: {decrypted_participant_id}")


Initial MD5 Hash Input (Hashed Participant ID): 76d4110e944e83212bafa4b11ebf2b7e
Decrypted Participant ID: 76d4110e944e83212bafa4b11ebf2b7e


# Unix epoch timestamp 

In [62]:
import time

timestamp = time.time()
print(timestamp)

1746851964.996321


In [63]:
import time

timestamp = time.time()
print(timestamp)


1746851965.0065382


In [64]:
import datetime

timestamp = 1743886473.0361397

dt_object = datetime.datetime.fromtimestamp(timestamp)
print(dt_object)

2025-04-05 13:54:33.036140
